# Testing the base Model (en-fr) in the test_dataset (fr)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Configuring the environment

In [2]:
! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [3]:
!pip install -q sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 18.6 MB/s eta 0:00:00


In [4]:
import os
os.environ["WANDB_DISABLED"]="true"

In [5]:
import os
import random
from datasets import Dataset, DatasetDict
from transformers import MarianTokenizer
from transformers import MarianMTModel, MarianTokenizer
import torch
from tqdm import tqdm

In [8]:
# Configuration
en_path = '/content/drive/MyDrive/Machine_Translation/POSTER/JRC-Acquis.en-fr.clean.en'
fr_path = '/content/drive/MyDrive/Machine_Translation/POSTER/JRC-Acquis.en-fr.clean.fr'
model_name = "Helsinki-NLP/opus-mt-en-fr"
max_length = 128
random.seed(42)

# Read and clean the files
with open(en_path, 'r', encoding='utf-8') as f_en, open(fr_path, 'r', encoding='utf-8') as f_fr:
    en_lines = f_en.readlines()
    fr_lines = f_fr.readlines()

pairs = [(en.strip(), fr.strip()) for en, fr in zip(en_lines, fr_lines) if en.strip() and fr.strip()]
random.shuffle(pairs)

# Split into train/dev/test
train_pairs = pairs[:80_000]
dev_pairs = pairs[80_000:90_000]
test_pairs = pairs[90_000:100_000]

# Create Hugging Face Datasets
def make_dataset(pairs):
    return Dataset.from_list([{"translation": {"en": en, "fr": fr}} for en, fr in pairs])

dataset = DatasetDict({
    "train": make_dataset(train_pairs),
    "validation": make_dataset(dev_pairs),
    "test": make_dataset(test_pairs)
})

# Tokenization
tokenizer = MarianTokenizer.from_pretrained(model_name)


def preprocess(examples):
    src_texts = [ex["en"] for ex in examples["translation"]]
    tgt_texts = [ex["fr"] for ex in examples["translation"]]

    model_inputs = tokenizer(src_texts, max_length=max_length, padding="max_length", truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(tgt_texts, max_length=max_length, padding="max_length", truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_dataset = dataset.map(preprocess, batched=True)


print("Done: tokenized_dataset['train'], ['validation'], ['test'] are ready.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

...

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Done: tokenized_dataset['train'], ['validation'], ['test'] are ready.


In [ ]:
# Load the fine-tuned model and tokenizer
model_dir = "Helsinki-NLP/opus-mt-en-fr"
model = MarianMTModel.from_pretrained(model_dir)
tokenizer = MarianTokenizer.from_pretrained(model_dir)
model.eval()
model.to("cuda" if torch.cuda.is_available() else "cpu")

# Extracting raw English and French sentences
raw_test_en = [ex["translation"]["en"] for ex in dataset["test"]]
raw_test_fr = [ex["translation"]["fr"] for ex in dataset["test"]]

# Generation of the translations with progress bar
translated_fr = []

for sentence in tqdm(raw_test_en, desc="Translating"):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(model.device)
    with torch.no_grad():
        generated_ids = model.generate(**inputs, max_length=128, num_beam=4)
    translated = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    translated_fr.append(translated)

Translating: 100%|██████████| 10000/10000 [54:27<00:00,  3.06it/s]


In [ ]:
import evaluate

bleu = evaluate.load("sacrebleu")
results = bleu.compute(predictions=translated_fr, references=[[ref] for ref in raw_test_fr])
print("BLEU score on test set:", results["score"])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.7 MB/s eta 0:00:00


BLEU score on test set: 58.52396979581673
